In [ ]:
import pandas as pd
import json
from geopy.distance import geodesic
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from scipy import stats

In [ ]:
file_path_business = "/content/yelp_academic_dataset_business (3).csv"
file_path_category = "/content/yelp_academic_dataset_category (3).csv"
file_path_user = "/content/yelp_academic_dataset_user (3).csv"
# file_path_review = "/content/yelp_academic_dataset_review_notext (2).csv"
file_path_hotels = '/content/Google Hotels.csv'
file_path_rental = '/content/Google Hotels Rental API FL.xlsx'
df_business = pd.read_csv(file_path_business, on_bad_lines='skip')
df_category = pd.read_csv(file_path_category, on_bad_lines='skip')
df_user = pd.read_csv(file_path_user, on_bad_lines='skip')
# df_review = pd.read_csv(file_path_review, on_bad_lines='skip')
df_hotel = pd.read_csv(file_path_hotels)
df_rental = pd.read_excel(file_path_rental)

# Hotel Data

In [ ]:
def expand_ratings(rating_str):
    try:
        # Convert the string to a list of dictionaries
        ratings = json.loads(rating_str.replace("'", '"'))
        # Create a dictionary with star ratings as keys and counts as values
        expanded = {f"star_{rating['stars']}": rating['count'] for rating in ratings}
        return expanded
    except Exception as e:
        print(f"Error processing row: {e}")
        return {}

# Apply the function to each row in the 'ratings' column
expanded_ratings = df_hotel['Ratings'].apply(expand_ratings)

# Convert the expanded ratings into separate columns and concatenate them back
expanded_df = pd.DataFrame(expanded_ratings.tolist())

# Concatenate the expanded DataFrame with the original DataFrame (if you want to keep original data)
df_hotel_expanded = pd.concat([df_hotel, expanded_df], axis=1)

In [ ]:
df_hotel_expanded = df_hotel_expanded.rename(columns={
    'star_1': 'Star 1',
    'star_2': 'Star 2',
    'star_3': 'Star 3',
    'star_4': 'Star 4',
    'star_5': 'Star 5'
})
# df_hotel_expanded.head()

In [ ]:
merged_df = pd.merge(df_business, df_category, on='business_id', how='inner')

# Now filter the merged DataFrame for Florida restaurants
florida_restaurants = merged_df[
    (merged_df['state'] == 'FL') &
    (merged_df['category_name'] == 'Restaurants')
]

In [ ]:
# Define a function to extract zip code using regex
def extract_zipcode(address):
    # Regex pattern for U.S. zip code (5 digits)
    match = re.search(r'\b\d{5}\b', address)
    if match:
        return match.group(0)  # Return the matched zip code
    return None  # Return None if no match is found

# Apply the function to the 'address' column of the hotels DataFrame
df_hotel_expanded['zipcode'] = df_hotel_expanded['Address'].apply(extract_zipcode)

# Show the first few rows to verify
# df_hotel_expanded[['Address', 'zipcode']].head(20)

In [ ]:
df_hotel_expanded.columns

Index(['Name', 'Description', 'Coordinates', 'Latitude', 'Longitude',
       'Hotel Class', 'Link', 'Ratings', 'Reviews', 'Reviews Breakdown',
       'Address', 'Star 5', 'Star 4', 'Star 3', 'Star 2', 'Star 1', 'zipcode'],
      dtype='object')

In [ ]:
florida_restaurants['postal_code'] = florida_restaurants['postal_code'].astype(str)
df_hotel_expanded['zipcode'] = df_hotel_expanded['zipcode'].astype(str)
florida_restaurants.rename(columns={'name': 'restaurant name'}, inplace=True)
df_hotel_expanded.rename(columns={'Name': 'Hotel Name'}, inplace=True)

<ipython-input-10-c2ecb839d040>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  florida_restaurants['postal_code'] = florida_restaurants['postal_code'].astype(str)
<ipython-input-10-c2ecb839d040>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  florida_restaurants.rename(columns={'name': 'restaurant name'}, inplace=True)


In [ ]:
florida_restaurants.count()

,0
bid,8731
business_id,8731
restaurant name,8731
address,8635
city,8731
state,8731
postal_code,8731
latitude,8731
longitude,8731
stars,8731


In [ ]:
# Merge the two tables using zipcode
matched_zip_df = pd.merge(florida_restaurants, df_hotel_expanded, left_on='postal_code', right_on='zipcode', how='inner')

# Define a function to calculate the distance between two geographical points (latitude and longitude)
def calculate_distance(row):
    restaurant_coords = (row['latitude'], row['longitude'])
    hotel_coords = (row['Latitude'], row['Longitude'])
    return geodesic(restaurant_coords, hotel_coords).kilometers

# Apply the function to calculate distances for each pair of matching restaurants and hotels
matched_zip_df['distance_km'] = matched_zip_df.apply(calculate_distance, axis=1)

matched_zip_df[['restaurant name', 'Hotel Name', 'postal_code', 'distance_km', 'Star 1', 'Star 2', 'Star 3', 'Star 4', 'Star 5']].head()

,restaurant name,Hotel Name,postal_code,distance_km,Star 1,Star 2,Star 3,Star 4,Star 5
0,Vietnamese Food Truck,Aloft Tampa Downtown,33602,1.002244,133,77,145,375,787
1,Vietnamese Food Truck,"Hotel Flor Tampa Downtown, Tapestry Collection...",33602,0.421970,230,110,158,329,829
2,Vietnamese Food Truck,JW Marriott Tampa Water Street,33602,1.622866,53,14,28,117,873
3,Vietnamese Food Truck,"Le MÃ©ridien Tampa, The Courthouse",33602,0.590933,64,34,102,261,837
4,Vietnamese Food Truck,Tampa Marriott Water Street,33602,1.622182,172,105,289,1163,3770


In [ ]:
matched_zip_df.columns

Index(['bid', 'business_id', 'restaurant name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'cid', 'category_name', 'Hotel Name', 'Description',
       'Coordinates', 'Latitude', 'Longitude', 'Hotel Class', 'Link',
       'Ratings', 'Reviews', 'Reviews Breakdown', 'Address', 'Star 5',
       'Star 4', 'Star 3', 'Star 2', 'Star 1', 'zipcode', 'distance_km'],
      dtype='object')

In [ ]:
df_correlation = pd.DataFrame(columns=['distance_category', 'stars', 'average', 'thumbup'])

In [ ]:
# Define the bins for distance
bins = [0, 1, 5, 10, float('inf')]  # Bins: 0-1, 1-5, 5-10, and greater than 10 km

# Define the labels for each bin
labels = ['less than 1 km', '1-5 km', '5-10 km', 'larger than 10 km']  # Labels for the bins

df_correlation['distance_category'] = pd.cut(matched_zip_df['distance_km'], bins=bins, labels=labels, right=False)

df_correlation['stars'] = matched_zip_df['stars']

df_correlation['average'] = ((matched_zip_df['Star 1'] + 2 * matched_zip_df['Star 2'] + 3 * matched_zip_df['Star 3']
                             + 4 * matched_zip_df['Star 4'] + 5 * matched_zip_df['Star 5'])
                             / (matched_zip_df['Star 1'] + matched_zip_df['Star 2'] + matched_zip_df['Star 3']
                             + matched_zip_df['Star 4'] + matched_zip_df['Star 5']))
df_correlation['thumbup'] = ((matched_zip_df['Star 4'] + matched_zip_df['Star 5'])
                             / (matched_zip_df['Star 1'] + matched_zip_df['Star 2'] + matched_zip_df['Star 3']
                             + matched_zip_df['Star 4'] + matched_zip_df['Star 5']))

In [ ]:
df_correlation.head()

,distance_category,stars,average,thumbup
0,1-5 km,4.0,4.058668,0.765985
1,less than 1 km,4.0,3.855676,0.699275
2,1-5 km,4.0,4.606452,0.912442
3,less than 1 km,4.0,4.365948,0.845917
4,1-5 km,4.0,4.501000,0.897072


In [ ]:
model = smf.ols(formula='stars ~ average + C(distance_category)', data=df_correlation).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     15.23
Date:                Tue, 03 Dec 2024   Prob (F-statistic):           2.01e-12
Time:                        03:28:09   Log-Likelihood:                -12117.
No. Observations:               10542   AIC:                         2.424e+04
Df Residuals:                   10537   BIC:                         2.428e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     3.2047      0.080     40.130      0.000       3.048       3.361
C(distance_category)[T.1-5 km]               -0.0255      0.017     -1.494      0.135      -0.059       0.008
C(distance_category)[T.5-10 km]               0.0007      0.025      0.029      0.977      -0.049       0.050
C(distance_category)[T.larger than 10 km]     0.2945      0.061      4.862      0.000       0.176       0.413
average                                       0.1057      0.019      5.670      0.000       0.069       0.142
==============================================================================
Omnibus:                      630.253   Durbin-Watson:                   0.655
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              747.129
Skew:                          -0.645   Prob(JB):                    5.79e-163
Kurtosis:                       3.196   Cond. No.                         48.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Randomly Extract 5 Distance in Each Range

In [ ]:
# Step 1: Randomly sample 5 values from each category (distance category)
random_samples_per_bin = df_correlation.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))

# Step 2: Reset the index for clarity
random_samples_per_bin = random_samples_per_bin.reset_index(drop=True)

<ipython-input-129-1d15cc3d9537>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  random_samples_per_bin = df_correlation.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))
<ipython-input-129-1d15cc3d9537>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  random_samples_per_bin = df_correlation.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))


In [ ]:
# Randomly Extract 5 Distance in Each Range
model = smf.ols(formula='stars ~ average + C(distance_category)', data=random_samples_per_bin).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                 -0.108
Method:                 Least Squares   F-statistic:                    0.5375
Date:                Tue, 03 Dec 2024   Prob (F-statistic):              0.710
Time:                        03:38:36   Log-Likelihood:                -17.494
No. Observations:                  20   AIC:                             44.99
Df Residuals:                      15   BIC:                             49.97
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     4.6007      2.121      2.169      0.047       0.080       9.121
C(distance_category)[T.1-5 km]               -0.2401      0.452     -0.531      0.603      -1.203       0.723
C(distance_category)[T.5-10 km]               0.2936      0.424      0.692      0.499      -0.610       1.198
C(distance_category)[T.larger than 10 km]     0.0316      0.432      0.073      0.943      -0.889       0.952
average                                      -0.1987      0.521     -0.381      0.708      -1.309       0.912
==============================================================================
Omnibus:                        3.404   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.182   Jarque-Bera (JB):                1.808
Skew:                          -0.708   Prob(JB):                        0.405
Kurtosis:                       3.408   Cond. No.                         62.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Short Term Rental Data

In [ ]:
# Define a function to extract zip code using regex
def extract_zipcode_rental(address):
    # Regex pattern for U.S. zip code (5 digits)
    match = re.search(r'\b\d{5}\b', address)
    if match:
        return match.group(0)  # Return the matched zip code
    return None  # Return None if no match is found

# Apply the function to the 'address' column of the hotels DataFrame
df_rental['zipcode'] = df_rental['Address'].apply(extract_zipcode_rental)

# Show the first few rows to verify
df_rental.head(20)

,Name,Coordinates,Latitude,Longitude,Link,Ratings,Reviews,Address,zipcode
0,"""Riverbend Gem"" - Studio|Courtyard|TPA Heights","{'latitude': 28.001779556274414, 'longitude': ...",28.001780,-82.465950,https://www.staytealdoor.com/properties/6621a3...,3.3,3,"6002, North Dexter Avenue, Kenilworth Grove, T...",33604
1,Creek Cabin - Two-Bedroom House,"{'latitude': 28.21503257751465, 'longitude': -...",28.215033,-82.698647,https://www.google.com/travel/search?q=7008%2C...,3.6,7,"7008, Creek Drive, Pasco County, Florida, 3465...",34655
2,TownePlace Suites by Marriott Tampa North/I-75...,"{'latitude': 28.067001342773438, 'longitude': ...",28.067001,-82.371696,https://deals.vio.com?sig=73aca13c7f952d2641c1...,3.6,189,"6814, Woodstork Drive, Temple Terrace, Hillsbo...",33637
3,BayCation,"{'latitude': 28.048564910888672, 'longitude': ...",28.048565,-82.401100,https://www.vacasa.com/unit/118653,3.8,4,"5025, East 110th Avenue, Tampa, Hillsborough C...",33617
4,Sunny Studio,"{'latitude': 27.776987075805664, 'longitude': ...",27.776987,-82.690620,https://cohostalbookings.hospitable.rentals/pr...,3.8,8,"4216, 5th Avenue North, Central Oak Park, Sain...",33713
5,Tropical Vibe,"{'latitude': 28.02916145324707, 'longitude': -...",28.029161,-82.743187,https://www.vacasa.com/unit/107891,3.8,11,"2299, Colonial Drive, Pinellas County, Florida...",34698
6,300 ftÂ² Private room âˆ™ 1 bedroom âˆ™ 2 guests,"{'latitude': 28.266399383544922, 'longitude': ...",28.266399,-82.725151,https://www.google.com/travel/search?q=5400%2C...,4.0,1,"5400, Bellview Avenue, New Port Richey, Pasco ...",34652
7,448 Little Harbor,"{'latitude': 27.725954055786133, 'longitude': ...",27.725954,-82.473824,https://www.google.com/travel/search?q=448%2C%...,4.0,20,"448, Bahia Beach Boulevard, Hillsborough Count...",33570
8,Land's End 203 building 1 Cute Coastal and Com...,"{'latitude': 27.7414608001709, 'longitude': -8...",27.741461,-82.756218,https://www.google.com/travel/search?q=7553%2C...,4.0,48,"7553, Bayshore Drive, Sunset Beach, Treasure I...",33706
9,"Villa KailÄsa - Pool, 6min drive to the Beach","{'latitude': 27.823320388793945, 'longitude': ...",27.823320,-82.778641,https://www.google.com/travel/search?q=5636%2C...,4.0,7,"5636, 100th Way North, Seminole, Pinellas Coun...",33708


In [ ]:
df_rental.rename(columns={'Ratings': 'rental_ratings'}, inplace=True)
df_rental.rename(columns={'Name': 'Rental Name'}, inplace=True)

df_rental.head()

,Rental Name,Coordinates,Latitude,Longitude,Link,rental_ratings,Reviews,Address,zipcode
0,"""Riverbend Gem"" - Studio|Courtyard|TPA Heights","{'latitude': 28.001779556274414, 'longitude': ...",28.001780,-82.465950,https://www.staytealdoor.com/properties/6621a3...,3.3,3,"6002, North Dexter Avenue, Kenilworth Grove, T...",33604
1,Creek Cabin - Two-Bedroom House,"{'latitude': 28.21503257751465, 'longitude': -...",28.215033,-82.698647,https://www.google.com/travel/search?q=7008%2C...,3.6,7,"7008, Creek Drive, Pasco County, Florida, 3465...",34655
2,TownePlace Suites by Marriott Tampa North/I-75...,"{'latitude': 28.067001342773438, 'longitude': ...",28.067001,-82.371696,https://deals.vio.com?sig=73aca13c7f952d2641c1...,3.6,189,"6814, Woodstork Drive, Temple Terrace, Hillsbo...",33637
3,BayCation,"{'latitude': 28.048564910888672, 'longitude': ...",28.048565,-82.401100,https://www.vacasa.com/unit/118653,3.8,4,"5025, East 110th Avenue, Tampa, Hillsborough C...",33617
4,Sunny Studio,"{'latitude': 27.776987075805664, 'longitude': ...",27.776987,-82.690620,https://cohostalbookings.hospitable.rentals/pr...,3.8,8,"4216, 5th Avenue North, Central Oak Park, Sain...",33713


In [ ]:
# Merge the two tables using zipcode
matched_rental_df = pd.merge(florida_restaurants, df_rental, left_on='postal_code', right_on='zipcode', how='inner')

# Define a function to calculate the distance between two geographical points (latitude and longitude)
def calculate_distance(row):
    restaurant_coords = (row['latitude'], row['longitude'])
    rental_coords = (row['Latitude'], row['Longitude'])
    return geodesic(restaurant_coords, rental_coords).kilometers

# Apply the function to calculate distances for each pair of matching restaurants and hotels
matched_rental_df['distance_km'] = matched_rental_df.apply(calculate_distance, axis=1)

matched_rental_df[['restaurant name', 'Rental Name', 'postal_code', 'distance_km', 'stars', 'rental_ratings']].head()

,restaurant name,Rental Name,postal_code,distance_km,stars,rental_ratings
0,The Pearl,Land's End 203 building 1 Cute Coastal and Com...,33706,3.283853,4.0,4.0
1,The Pearl,Sunrise Resort Unit 502,33706,4.620419,4.0,4.7
2,Trust Me BBQ,Discover Your Home Away from Home in Riverview,33569,0.875711,4.0,5.0
3,Trust Me BBQ,"Entire home just outside Tampa, FL",33569,2.269097,4.0,5.0
4,PDQ Temple Terrace,BayCation,33617,0.717325,3.0,3.8


In [ ]:
matched_rental_df.to_csv('matched_rental_df.csv', index=False)

In [ ]:
df_rental.head()

,Rental Name,Coordinates,Latitude,Longitude,Link,rental_ratings,Reviews,Address,zipcode
0,"""Riverbend Gem"" - Studio|Courtyard|TPA Heights","{'latitude': 28.001779556274414, 'longitude': ...",28.001780,-82.465950,https://www.staytealdoor.com/properties/6621a3...,3.3,3,"6002, North Dexter Avenue, Kenilworth Grove, T...",33604
1,Creek Cabin - Two-Bedroom House,"{'latitude': 28.21503257751465, 'longitude': -...",28.215033,-82.698647,https://www.google.com/travel/search?q=7008%2C...,3.6,7,"7008, Creek Drive, Pasco County, Florida, 3465...",34655
2,TownePlace Suites by Marriott Tampa North/I-75...,"{'latitude': 28.067001342773438, 'longitude': ...",28.067001,-82.371696,https://deals.vio.com?sig=73aca13c7f952d2641c1...,3.6,189,"6814, Woodstork Drive, Temple Terrace, Hillsbo...",33637
3,BayCation,"{'latitude': 28.048564910888672, 'longitude': ...",28.048565,-82.401100,https://www.vacasa.com/unit/118653,3.8,4,"5025, East 110th Avenue, Tampa, Hillsborough C...",33617
4,Sunny Studio,"{'latitude': 27.776987075805664, 'longitude': ...",27.776987,-82.690620,https://cohostalbookings.hospitable.rentals/pr...,3.8,8,"4216, 5th Avenue North, Central Oak Park, Sain...",33713


In [ ]:
df_correlation_rental = pd.DataFrame(columns=['distance_category', 'stars', 'rental_ratings'])

# Define the bins for distance
bins = [0, 1, 5, 10, float('inf')]  # Bins: 0-1, 1-5, 5-10, and greater than 10 km

# Define the labels for each bin
labels = ['less than 1 km', '1-5 km', '5-10 km', 'larger than 10 km']  # Labels for the bins

df_correlation_rental['distance_category'] = pd.cut(matched_rental_df['distance_km'], bins=bins, labels=labels, right=False)

df_correlation_rental['stars'] = matched_rental_df['stars']
df_correlation_rental['rental_ratings'] = matched_rental_df['rental_ratings']
df_correlation_rental

,distance_category,stars,rental_ratings
0,1-5 km,4.0,4.0
1,1-5 km,4.0,4.7
2,less than 1 km,4.0,5.0
3,1-5 km,4.0,5.0
4,less than 1 km,3.0,3.8
...,...,...,...
11501,1-5 km,3.5,4.0
11502,less than 1 km,3.5,5.0
11503,1-5 km,3.5,5.0
11504,1-5 km,3.5,5.0


In [ ]:
model = smf.ols(formula='stars ~ rental_ratings + C(distance_category)', data=df_correlation_rental).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     18.70
Date:                Tue, 03 Dec 2024   Prob (F-statistic):           2.48e-15
Time:                        03:29:14   Log-Likelihood:                -13861.
No. Observations:               11506   AIC:                         2.773e+04
Df Residuals:                   11501   BIC:                         2.777e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     4.1765      0.097     43.177      0.000       3.987       4.366
C(distance_category)[T.1-5 km]               -0.0732      0.022     -3.365      0.001      -0.116      -0.031
C(distance_category)[T.5-10 km]              -0.1928      0.031     -6.302      0.000      -0.253      -0.133
C(distance_category)[T.larger than 10 km]     0.0408      0.092      0.442      0.659      -0.140       0.222
rental_ratings                               -0.1111      0.020     -5.456      0.000      -0.151      -0.071
==============================================================================
Omnibus:                      555.117   Durbin-Watson:                   0.676
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              635.251
Skew:                          -0.572   Prob(JB):                    1.14e-138
Kurtosis:                       2.869   Cond. No.                         63.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Step 1: Randomly sample 5 values from each category (distance category)
random_samples_per_bin = df_correlation_rental.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))

# Step 2: Reset the index for clarity
random_samples_per_bin = random_samples_per_bin.reset_index(drop=True)

<ipython-input-133-65e4a84ddf2d>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  random_samples_per_bin = df_correlation_rental.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))
<ipython-input-133-65e4a84ddf2d>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  random_samples_per_bin = df_correlation_rental.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))


In [ ]:
model = smf.ols(formula='stars ~ rental_ratings + C(distance_category)', data=random_samples_per_bin).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                 -0.042
Method:                 Least Squares   F-statistic:                    0.8082
Date:                Tue, 03 Dec 2024   Prob (F-statistic):              0.539
Time:                        03:39:22   Log-Likelihood:                -22.310
No. Observations:                  20   AIC:                             54.62
Df Residuals:                      15   BIC:                             59.60
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     4.1370      2.562      1.615      0.127      -1.324       9.598
C(distance_category)[T.1-5 km]                0.6203      0.542      1.145      0.270      -0.535       1.775
C(distance_category)[T.5-10 km]               0.7243      0.543      1.334      0.202      -0.433       1.882
C(distance_category)[T.larger than 10 km]     0.9406      0.550      1.710      0.108      -0.232       2.113
rental_ratings                               -0.2028      0.548     -0.370      0.717      -1.372       0.966
==============================================================================
Omnibus:                        5.187   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.075   Jarque-Bera (JB):                3.004
Skew:                          -0.874   Prob(JB):                        0.223
Kurtosis:                       3.741   Cond. No.                         66.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# API Price Level

In [ ]:
# Link for API to extract price level:
# https://colab.research.google.com/drive/1E8gfRpXtURdHon4BbXBy9budVkToFKBL?usp=sharing#scrollTo=7WVRdaOCqHva

In [ ]:
df_pricelevel = pd.read_excel('/content/pricelevels_final.xlsx')

In [ ]:
df_pricelevel.head()

,name,price
0,Vietnamese Food Truck,NaN
1,Cesarina's Italian Deli,$
2,Roman Forum,NaN
3,Pizza Joe's - a Taste of Buffalo,NaN
4,Top Shelf Sports Lounge,$$


In [ ]:
florida_restaurants.head()

,bid,business_id,restaurant name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,cid,category_name
56,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,10,0,57,Restaurants
70,15,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.91612,-82.76046,4.5,100,0,71,Restaurants
275,59,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.04620,-82.50505,4.0,23,0,276,Restaurants
278,60,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.96051,-82.50613,4.0,35,0,279,Restaurants
360,80,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.19625,-82.38062,4.5,95,0,361,Restaurants


In [ ]:
merged_florida_restaurants = pd.merge(florida_restaurants, df_pricelevel,
                                      left_on='restaurant name',
                                      right_on='name',
                                      how='left',
                                      suffixes=('_df1', '_df2'))
merged_florida_restaurants

,bid,business_id,restaurant name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,cid,category_name,name,price
0,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,10,0,57,Restaurants,Vietnamese Food Truck,NaN
1,15,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.91612,-82.76046,4.5,100,0,71,Restaurants,NaN,NaN
2,59,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.04620,-82.50505,4.0,23,0,276,Restaurants,Roman Forum,NaN
3,60,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.96051,-82.50613,4.0,35,0,279,Restaurants,NaN,NaN
4,80,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.19625,-82.38062,4.5,95,0,361,Restaurants,Top Shelf Sports Lounge,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76633,150250,8MzF1Tlgz0pOkxmhP5dYzA,El Cap Restaurant,3500 4th St N,St. Petersburg,FL,33704,27.80414,-82.63885,3.5,414,0,668163,Restaurants,El Cap Restaurant,$
76634,150263,-bZQH8yjm7ntTyGeLQwh8Q,Farmer's Kitchen Restaurant,3500 E Bay Dr,Largo,FL,33771,27.91679,-82.75039,4.0,6,0,668223,Restaurants,NaN,NaN
76635,150272,BIyT7Kr7tMJqlfp4oOOYQg,Copper Bell Cafe,11228 Boyette Rd,Riverview,FL,33569,27.85374,-82.31689,3.5,49,0,668269,Restaurants,NaN,NaN
76636,150293,esBGrrmuZzSiECyRBoKvvA,Colony Grill - St. Petersburg,670 Central Ave,St. Petersburg,FL,33701,27.77087,-82.64307,4.5,38,0,668358,Restaurants,Colony Grill - St. Petersburg,$$


In [ ]:
print(florida_restaurants['business_id'].duplicated().sum())

0


In [ ]:
print(merged_florida_restaurants['business_id'].duplicated().sum())

67907


In [ ]:
merged_florida_restaurants = merged_florida_restaurants.drop_duplicates(subset='business_id')

In [ ]:
merged_florida_restaurants.head()

,bid,business_id,restaurant name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,cid,category_name,name,price
0,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,10,0,57,Restaurants,Vietnamese Food Truck,NaN
1,15,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.91612,-82.76046,4.5,100,0,71,Restaurants,NaN,NaN
2,59,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.04620,-82.50505,4.0,23,0,276,Restaurants,Roman Forum,NaN
3,60,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.96051,-82.50613,4.0,35,0,279,Restaurants,NaN,NaN
4,80,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.19625,-82.38062,4.5,95,0,361,Restaurants,Top Shelf Sports Lounge,$$


In [ ]:
merged_florida_restaurants.count()

,0
bid,8731
business_id,8731
restaurant name,8731
address,8635
city,8731
state,8731
postal_code,8731
latitude,8731
longitude,8731
stars,8731


In [ ]:
price_map = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}

# Replace the symbols in the 'price_category' column with their corresponding numerical values
merged_florida_restaurants['price'] = merged_florida_restaurants['price'].replace(price_map)

<ipython-input-36-d80f6da20237>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_florida_restaurants['price'] = merged_florida_restaurants['price'].replace(price_map)
<ipython-input-36-d80f6da20237>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_florida_restaurants['price'] = merged_florida_restaurants['price'].replace(price_map)


In [ ]:
merged_florida_restaurants.head()

,bid,business_id,restaurant name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,cid,category_name,name,price
0,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,10,0,57,Restaurants,Vietnamese Food Truck,NaN
1,15,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.91612,-82.76046,4.5,100,0,71,Restaurants,NaN,NaN
2,59,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.04620,-82.50505,4.0,23,0,276,Restaurants,Roman Forum,NaN
3,60,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.96051,-82.50613,4.0,35,0,279,Restaurants,NaN,NaN
4,80,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.19625,-82.38062,4.5,95,0,361,Restaurants,Top Shelf Sports Lounge,2.0


In [ ]:
df_pricelevel

,name,price
0,Vietnamese Food Truck,NaN
1,Cesarina's Italian Deli,$
2,Roman Forum,NaN
3,Pizza Joe's - a Taste of Buffalo,NaN
4,Top Shelf Sports Lounge,$$
...,...,...
8627,First Watch,$$
8628,El Cap Restaurant,$
8629,Benedict's Family Restaurant,$$
8630,Oxford Exchange,$$


In [ ]:
matched_zip_df

,bid,business_id,restaurant name,address,city,state,postal_code,latitude,longitude,stars,...,Reviews,Reviews Breakdown,Address,Star 5,Star 4,Star 3,Star 2,Star 1,zipcode,distance_km
0,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,1517,"[{'name': 'Bar', 'description': 'Bar or lounge...","Kennedy Boulevard @ Ashley Drive, 100, West Ke...",787,375,145,77,133,33602,1.002244
1,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,1656,"[{'name': 'Service', 'description': 'Service',...","Hotel Flor Tampa Downtown, Tapestry Collection...",829,329,158,110,230,33602,0.421970
2,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,1085,"[{'name': 'Wellness', 'description': 'Wellness...","Tampa Marriott Water Street, 505, Water Street...",873,117,28,14,53,33602,1.622866
3,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,1298,"[{'name': 'Couples', 'description': 'Couple fr...","Le MÃ©ridien Tampa, 601, North Florida Avenue,...",837,261,102,34,64,33602,0.590933
4,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,5499,"[{'name': 'Location', 'description': 'Location...","Tampa Marriott Water Street, 505, Water Street...",3770,1163,289,105,172,33602,1.622182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10537,150202,xGcpC8D4Sio-bN9KCr054g,Zen Bistro Grill + Sushi,2223 N Westshore Blvd,Tampa,FL,33607,27.96405,-82.52160,3.5,...,3636,"[{'name': 'Nature', 'description': 'Nature and...","Grand Hyatt Tampa Bay, 2900, Bayport Drive, Ta...",2596,670,168,76,126,33607,2.805708
10538,150202,xGcpC8D4Sio-bN9KCr054g,Zen Bistro Grill + Sushi,2223 N Westshore Blvd,Tampa,FL,33607,27.96405,-82.52160,3.5,...,1754,"[{'name': 'Service', 'description': 'Service',...","Marriott Tampa Westshore, 1001, North West Sho...",1012,449,156,54,83,33607,1.220064
10539,150202,xGcpC8D4Sio-bN9KCr054g,Zen Bistro Grill + Sushi,2223 N Westshore Blvd,Tampa,FL,33607,27.96405,-82.52160,3.5,...,2499,"[{'name': 'Nature', 'description': 'Nature and...","Sailport Waterfront Suites, 2506, North Rocky ...",1413,597,213,110,166,33607,4.949577
10540,150202,xGcpC8D4Sio-bN9KCr054g,Zen Bistro Grill + Sushi,2223 N Westshore Blvd,Tampa,FL,33607,27.96405,-82.52160,3.5,...,4051,"[{'name': 'Nature', 'description': 'Nature and...","The Godfrey Hotel & Cabanas Tampa, 7700, Bay H...",1549,810,569,360,763,33607,4.826658


In [ ]:
# Merge the two DataFrames on 'restaurant_id'
merged_hotel_distance = pd.merge(matched_zip_df, df_pricelevel, left_on='restaurant name', right_on='name', how='inner')

# Define the price map for transforming the price category
price_map = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}

# Apply the price map to the 'price_category' column
merged_hotel_distance['price'] = merged_hotel_distance['price'].replace(price_map)

# Print the merged DataFrame
print(merged_hotel_distance)

          bid             business_id             restaurant name  \
0          12  eEOYSgkmpB90uNA7lDOMRA       Vietnamese Food Truck   
1          12  eEOYSgkmpB90uNA7lDOMRA       Vietnamese Food Truck   
2          12  eEOYSgkmpB90uNA7lDOMRA       Vietnamese Food Truck   
3          12  eEOYSgkmpB90uNA7lDOMRA       Vietnamese Food Truck   
4          12  eEOYSgkmpB90uNA7lDOMRA       Vietnamese Food Truck   
...       ...                     ...                         ...   
55051  149526  eJxNnmDHBzYfxD5lFiifJw  Shaner's Land & Sea Market   
55052  149599  YaWt65tSRsyK6bppZUY-dg               Bernie's Deli   
55053  149599  YaWt65tSRsyK6bppZUY-dg               Bernie's Deli   
55054  149599  YaWt65tSRsyK6bppZUY-dg               Bernie's Deli   
55055  150158  s2eyoTuJrcP7I_XyjdhUHQ               Bros Pizzeria   

                       address            city state postal_code  latitude  \
0                          NaN       Tampa Bay    FL       33602  27.95527   
1              

<ipython-input-40-74c2ec10b7cd>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_hotel_distance['price'] = merged_hotel_distance['price'].replace(price_map)


In [ ]:
# Define the bins for distance
bins = [0, 1, 5, 10, float('inf')]  # Bins: 0-1, 1-5, 5-10, and greater than 10 km

# Define the labels for each bin
labels = ['less than 1 km', '1-5 km', '5-10 km', 'larger than 10 km']  # Labels for the bins

merged_hotel_distance['distance_category'] = pd.cut(matched_zip_df['distance_km'], bins=bins, labels=labels, right=False)
merged_hotel_distance['average'] = ((matched_zip_df['Star 1'] + 2 * matched_zip_df['Star 2'] + 3 * matched_zip_df['Star 3']
                             + 4 * matched_zip_df['Star 4'] + 5 * matched_zip_df['Star 5'])
                             / (matched_zip_df['Star 1'] + matched_zip_df['Star 2'] + matched_zip_df['Star 3']
                             + matched_zip_df['Star 4'] + matched_zip_df['Star 5']))

In [ ]:
merged_hotel_distance

,bid,business_id,restaurant name,address,city,state,postal_code,latitude,longitude,stars,...,Star 4,Star 3,Star 2,Star 1,zipcode,distance_km,name,price,distance_category,average
0,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,375,145,77,133,33602,1.002244,Vietnamese Food Truck,NaN,1-5 km,4.058668
1,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,329,158,110,230,33602,0.421970,Vietnamese Food Truck,NaN,less than 1 km,3.855676
2,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,117,28,14,53,33602,1.622866,Vietnamese Food Truck,NaN,1-5 km,4.606452
3,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,261,102,34,64,33602,0.590933,Vietnamese Food Truck,NaN,less than 1 km,4.365948
4,12,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.95527,-82.45632,4.0,...,1163,289,105,172,33602,1.622182,Vietnamese Food Truck,NaN,1-5 km,4.501000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55051,149526,eJxNnmDHBzYfxD5lFiifJw,Shaner's Land & Sea Market,2000 Pass A Grille Way,St. Pete Beach,FL,33706,27.69667,-82.73604,4.0,...,8,2,2,4,33706,4.756067,Shaner's Land & Sea Market,2.0,NaN,NaN
55052,149599,YaWt65tSRsyK6bppZUY-dg,Bernie's Deli,3625 Queen Palm Dr,Tampa,FL,33619,27.97497,-82.33729,2.5,...,210,89,34,70,33619,4.525001,Bernie's Deli,1.0,NaN,NaN
55053,149599,YaWt65tSRsyK6bppZUY-dg,Bernie's Deli,3625 Queen Palm Dr,Tampa,FL,33619,27.97497,-82.33729,2.5,...,183,88,31,57,33619,3.857208,Bernie's Deli,1.0,NaN,NaN
55054,149599,YaWt65tSRsyK6bppZUY-dg,Bernie's Deli,3625 Queen Palm Dr,Tampa,FL,33619,27.97497,-82.33729,2.5,...,49,12,9,27,33619,4.989128,Bernie's Deli,1.0,NaN,NaN


In [ ]:
model = smf.ols(formula='stars ~ average + price + C(distance_category)', data=merged_hotel_distance).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     974.5
Date:                Tue, 03 Dec 2024   Prob (F-statistic):               0.00
Time:                        03:29:50   Log-Likelihood:                -10059.
No. Observations:                9595   AIC:                         2.013e+04
Df Residuals:                    9589   BIC:                         2.017e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     1.1136      0.078     14.269      0.000       0.961       1.267
C(distance_category)[T.1-5 km]                0.0233      0.016      1.440      0.150      -0.008       0.055
C(distance_category)[T.5-10 km]               0.0725      0.024      3.040      0.002       0.026       0.119
C(distance_category)[T.larger than 10 km]     0.1191      0.057      2.089      0.037       0.007       0.231
average                                       0.0274      0.018      1.557      0.120      -0.007       0.062
price                                         1.1105      0.016     69.545      0.000       1.079       1.142
==============================================================================
Omnibus:                     1922.765   Durbin-Watson:                   0.210
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3916.237
Skew:                           1.193   Prob(JB):                         0.00
Kurtosis:                       5.026   Cond. No.                         51.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Step 1: Randomly sample 5 values from each category (distance category)
random_samples_per_bin = merged_hotel_distance.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))

# Step 2: Reset the index for clarity
random_samples_per_bin = random_samples_per_bin.reset_index(drop=True)

<ipython-input-136-c62cbd6f3843>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  random_samples_per_bin = merged_hotel_distance.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))
<ipython-input-136-c62cbd6f3843>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  random_samples_per_bin = merged_hotel_distance.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))


In [ ]:
model = smf.ols(formula='stars ~ average + price + C(distance_category)', data=random_samples_per_bin).fit()
model.summary()

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.445
Method:                 Least Squares   F-statistic:                     3.887
Date:                Tue, 03 Dec 2024   Prob (F-statistic):             0.0225
Time:                        03:42:31   Log-Likelihood:                -11.313
No. Observations:                  19   AIC:                             34.63
Df Residuals:                      13   BIC:                             40.29
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     1.1022      1.969      0.560      0.585      -3.152       5.357
C(distance_category)[T.1-5 km]               -0.3113      0.412     -0.756      0.463      -1.201       0.578
C(distance_category)[T.5-10 km]              -0.1844      0.349     -0.528      0.606      -0.938       0.570
C(distance_category)[T.larger than 10 km]     0.0095      0.378      0.025      0.980      -0.807       0.826
average                                       0.0972      0.446      0.218      0.831      -0.865       1.060
price                                         0.7812      0.227      3.447      0.004       0.292       1.271
==============================================================================
Omnibus:                        0.823   Durbin-Watson:                   2.554
Prob(Omnibus):                  0.663   Jarque-Bera (JB):                0.724
Skew:                           0.161   Prob(JB):                        0.696
Kurtosis:                       2.100   Cond. No.                         74.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
df_rental.head()

,Rental Name,Coordinates,Latitude,Longitude,Link,rental_ratings,Reviews,Address,zipcode
0,"""Riverbend Gem"" - Studio|Courtyard|TPA Heights","{'latitude': 28.001779556274414, 'longitude': ...",28.001780,-82.465950,https://www.staytealdoor.com/properties/6621a3...,3.3,3,"6002, North Dexter Avenue, Kenilworth Grove, T...",33604
1,Creek Cabin - Two-Bedroom House,"{'latitude': 28.21503257751465, 'longitude': -...",28.215033,-82.698647,https://www.google.com/travel/search?q=7008%2C...,3.6,7,"7008, Creek Drive, Pasco County, Florida, 3465...",34655
2,TownePlace Suites by Marriott Tampa North/I-75...,"{'latitude': 28.067001342773438, 'longitude': ...",28.067001,-82.371696,https://deals.vio.com?sig=73aca13c7f952d2641c1...,3.6,189,"6814, Woodstork Drive, Temple Terrace, Hillsbo...",33637
3,BayCation,"{'latitude': 28.048564910888672, 'longitude': ...",28.048565,-82.401100,https://www.vacasa.com/unit/118653,3.8,4,"5025, East 110th Avenue, Tampa, Hillsborough C...",33617
4,Sunny Studio,"{'latitude': 27.776987075805664, 'longitude': ...",27.776987,-82.690620,https://cohostalbookings.hospitable.rentals/pr...,3.8,8,"4216, 5th Avenue North, Central Oak Park, Sain...",33713


In [ ]:
# Merge the two DataFrames on 'restaurant_id'
merged_rental_distance = pd.merge(matched_rental_df, df_pricelevel, left_on='restaurant name', right_on='name', how='inner')

# Define the price map for transforming the price category
price_map = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}

# Apply the price map to the 'price_category' column
merged_rental_distance['price'] = merged_rental_distance['price'].replace(price_map)

# Print the merged DataFrame
print(merged_rental_distance)

          bid             business_id                restaurant name  \
0         107  vje0KIiE7vtpx7JzmBx5LQ                      The Pearl   
1         107  vje0KIiE7vtpx7JzmBx5LQ                      The Pearl   
2         276  dJfkfBbJz7wi0RgW-ph_Eg                     Sake House   
3         276  dJfkfBbJz7wi0RgW-ph_Eg                     Sake House   
4         276  dJfkfBbJz7wi0RgW-ph_Eg                     Sake House   
...       ...                     ...                            ...   
91809  150158  s2eyoTuJrcP7I_XyjdhUHQ                  Bros Pizzeria   
91810  150158  s2eyoTuJrcP7I_XyjdhUHQ                  Bros Pizzeria   
91811  150158  s2eyoTuJrcP7I_XyjdhUHQ                  Bros Pizzeria   
91812  150293  esBGrrmuZzSiECyRBoKvvA  Colony Grill - St. Petersburg   
91813  150293  esBGrrmuZzSiECyRBoKvvA  Colony Grill - St. Petersburg   

                       address             city state postal_code  latitude  \
0                163 107th Ave  Treasure Island    FL   

<ipython-input-44-986ecd13aa3e>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_rental_distance['price'] = merged_rental_distance['price'].replace(price_map)


In [ ]:
# Define the bins for distance
bins = [0, 1, 5, 10, float('inf')]  # Bins: 0-1, 1-5, 5-10, and greater than 10 km

# Define the labels for each bin
labels = ['less than 1 km', '1-5 km', '5-10 km', 'larger than 10 km']  # Labels for the bins

merged_rental_distance['distance_category'] = pd.cut(matched_zip_df['distance_km'], bins=bins, labels=labels, right=False)

In [ ]:
model = smf.ols(formula='stars ~ C(rental_ratings) + price + C(distance_category)', data=merged_rental_distance).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     201.1
Date:                Tue, 03 Dec 2024   Prob (F-statistic):               0.00
Time:                        03:30:06   Log-Likelihood:                -10714.
No. Observations:                9738   AIC:                         2.146e+04
Df Residuals:                    9720   BIC:                         2.159e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     0.7304      0.130      5.598      0.000       0.475       0.986
C(rental_ratings)[T.3.6]                      1.2515      0.161      7.776      0.000       0.936       1.567
C(rental_ratings)[T.3.8]                      1.1341      0.139      8.166      0.000       0.862       1.406
C(rental_ratings)[T.4.0]                      0.4187      0.135      3.097      0.002       0.154       0.684
C(rental_ratings)[T.4.1]                      1.5654      0.151     10.343      0.000       1.269       1.862
C(rental_ratings)[T.4.2]                      0.6840      0.142      4.834      0.000       0.407       0.961
C(rental_ratings)[T.4.3]                      0.4098      0.133      3.091      0.002       0.150       0.670
C(rental_ratings)[T.4.4]                      0.7718      0.135      5.737      0.000       0.508       1.035
C(rental_ratings)[T.4.5]                      0.5303      0.133      3.983      0.000       0.269       0.791
C(rental_ratings)[T.4.6]                      0.8998      0.131      6.869      0.000       0.643       1.157
C(rental_ratings)[T.4.7]                      0.7061      0.132      5.368      0.000       0.448       0.964
C(rental_ratings)[T.4.8]                      0.6596      0.131      5.037      0.000       0.403       0.916
C(rental_ratings)[T.4.9]                      0.5974      0.131      4.576      0.000       0.341       0.853
C(rental_ratings)[T.5.0]                      0.6074      0.129      4.700      0.000       0.354       0.861
C(distance_category)[T.1-5 km]               -0.0131      0.017     -0.774      0.439      -0.046       0.020
C(distance_category)[T.5-10 km]              -0.0602      0.025     -2.414      0.016      -0.109      -0.011
C(distance_category)[T.larger than 10 km]    -0.0276      0.061     -0.455      0.649      -0.146       0.091
price                                         0.9809      0.018     53.054      0.000       0.945       1.017
==============================================================================
Omnibus:                     1195.741   Durbin-Watson:                   0.157
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1707.626
Skew:                           0.943   Prob(JB):                         0.00
Kurtosis:                       3.805   Cond. No.                         115.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Step 1: Randomly sample 5 values from each category (distance category)
random_samples_per_bin = merged_rental_distance.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))

# Step 2: Reset the index for clarity
random_samples_per_bin = random_samples_per_bin.reset_index(drop=True)

<ipython-input-138-c30213338a5a>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  random_samples_per_bin = merged_rental_distance.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))
<ipython-input-138-c30213338a5a>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  random_samples_per_bin = merged_rental_distance.groupby('distance_category').apply(lambda x: x.sample(n=5, random_state=42))


In [ ]:
model = smf.ols(formula='stars ~ C(rental_ratings) + price + C(distance_category)', data=random_samples_per_bin).fit()
model.summary()

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     1.777
Date:                Tue, 03 Dec 2024   Prob (F-statistic):              0.248
Time:                        03:43:20   Log-Likelihood:                -6.0771
No. Observations:                  19   AIC:                             38.15
Df Residuals:                       6   BIC:                             50.43
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     1.6455      1.105      1.489      0.187      -1.058       4.349
C(rental_ratings)[T.4.0]                     -0.7788      1.050     -0.741      0.486      -3.349       1.792
C(rental_ratings)[T.4.4]                     -0.7788      1.050     -0.741      0.486      -3.349       1.792
C(rental_ratings)[T.4.5]                      0.0606      0.935      0.065      0.950      -2.226       2.347
C(rental_ratings)[T.4.6]                      0.7667      1.061      0.723      0.497      -1.829       3.362
C(rental_ratings)[T.4.7]                      1.0000      0.838      1.193      0.278      -1.052       3.052
C(rental_ratings)[T.4.8]                     -0.0242      0.756     -0.032      0.975      -1.873       1.825
C(rental_ratings)[T.4.9]                     -0.5061      0.909     -0.557      0.598      -2.731       1.719
C(rental_ratings)[T.5.0]                      1.0121      0.703      1.440      0.200      -0.708       2.732
C(distance_category)[T.1-5 km]               -0.7788      0.633     -1.231      0.265      -2.327       0.770
C(distance_category)[T.5-10 km]              -0.6788      0.510     -1.331      0.231      -1.926       0.569
C(distance_category)[T.larger than 10 km]    -0.5030      0.569     -0.884      0.411      -1.895       0.889
price                                         1.1333      0.530      2.137      0.076      -0.164       2.431
==============================================================================
Omnibus:                        1.167   Durbin-Watson:                   2.440
Prob(Omnibus):                  0.558   Jarque-Bera (JB):                0.474
Skew:                          -0.386   Prob(JB):                        0.789
Kurtosis:                       3.050   Cond. No.                         29.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Data Prep for Machine Learning Models

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Prepare features (X) and target (y)
X = merged_hotel_distance[['price', 'distance_km']]  # You can include more features if necessary
y = merged_hotel_distance['stars']  # The target variable (restaurant stars)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optionally, scale the features (important for some models like SVM or neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest Model

In [ ]:
# Initialize and train the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test_scaled)

# Evaluate the Random Forest model
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)

print(f"Random Forest Regressor - MSE: {rf_mse:.4f}, R-squared: {rf_r2:.4f}")

Random Forest Regressor - MSE: 0.0921, R-squared: 0.8876
